In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chocolate-rating-prediction-ai-edu/chocolate_train.csv
/kaggle/input/chocolate-rating-prediction-ai-edu/choco_sample_submission.csv
/kaggle/input/chocolate-rating-prediction-ai-edu/chocolate_test_new.csv


In [2]:
import warnings
warnings.filterwarnings('ignore')

Скачиваем данные для тренировки и изучаем их.

In [3]:
data = pd.read_csv('/kaggle/input/chocolate-rating-prediction-ai-edu/chocolate_train.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1255 entries, 0 to 1254
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Company               1255 non-null   object 
 1   Specific Bean Origin  1255 non-null   object 
 2   REF                   1255 non-null   int64  
 3   Review                1255 non-null   int64  
 4   Cocoa Percent         1255 non-null   object 
 5   Company Location      1255 non-null   object 
 6   Rating                1255 non-null   float64
 7   Bean Type             1254 non-null   object 
 8   Broad Bean Origin     1254 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 88.4+ KB


Видим, что имеются пропуски в признаках. Причем их очень мало, поэтому просто удалим объекты с пропусками.

In [4]:
data = data.dropna(subset=['Bean Type', 'Broad Bean Origin'])
y = data['Rating']
X = data.drop(columns=['Rating'], inplace=False)
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1253 entries, 0 to 1254
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Company               1253 non-null   object
 1   Specific Bean Origin  1253 non-null   object
 2   REF                   1253 non-null   int64 
 3   Review                1253 non-null   int64 
 4   Cocoa Percent         1253 non-null   object
 5   Company Location      1253 non-null   object
 6   Bean Type             1253 non-null   object
 7   Broad Bean Origin     1253 non-null   object
dtypes: int64(2), object(6)
memory usage: 88.1+ KB


Поменяем процент какао на числовой признак.

In [5]:
X['Cocoa Percent']
X['Cocoa Percent'] = X['Cocoa Percent'].map(lambda x: x[:-1])
X['Cocoa Percent'] = X['Cocoa Percent'].astype(dtype='float64')

In [6]:
!pip install association-metrics

  Preparing metadata (setup.py) ... done
  Created wheel for association-metrics: filename=association_metrics-0.0.1-py3-none-any.whl size=3928 sha256=e8f4a106de1b42870c7b491dee3a19091535d6d4ca087739e67162258beb47c9
  Stored in directory: /root/.cache/pip/wheels/01/47/5f/4360ef6988df6f1b0c1857cab43334269110160bd97b716936
Successfully built association-metrics


In [7]:
import association_metrics as am

XC = X.apply(
        lambda x: x.astype("category") if x.dtype == "object" else x)

cramersv = am.CramersV(XC)

cramersv.fit()

,Company,Specific Bean Origin,Company Location,Bean Type,Broad Bean Origin
Company,1.000000,0.771180,0.999651,0.616975,0.607303
Specific Bean Origin,0.771180,1.000000,0.808513,0.907951,0.983334
Company Location,0.999651,0.808513,1.000000,0.291667,0.383039
Bean Type,0.616975,0.907951,0.291667,1.000000,0.388294
Broad Bean Origin,0.607303,0.983334,0.383039,0.388294,1.000000


Мы видим, что некоторые признаки очень взаимосвязаны. Например компания и местоположение компании (что неудивительно). Избавимся от таких зависимостей. Конечно еще имеется сорт бобов, но его оставим, ибо он может оказаться важным но если оставить только его, то так как сортов меньше чем регионов мы потеряем в информативности

In [8]:
X.drop(columns=['Company Location', 'Specific Bean Origin'], inplace=True)

Cделаем тоже самое с числовыми признаками.

In [9]:
X[['REF', 'Review', 'Cocoa Percent']].corr()

,REF,Review,Cocoa Percent
REF,1.000000,0.984950,0.044877
Review,0.984950,1.000000,0.045225
Cocoa Percent,0.044877,0.045225,1.000000


Видим, что REF и Review сильно скоррелированы. Поэтому уберем один из этих признаков.

In [10]:
X.drop(columns=['Review'], inplace=True)


Обратотаем пропуски:

In [11]:
for c in X.columns:
    print(c, len(X[X[c] == '\xa0']))

Company 0
REF 0
Cocoa Percent 0
Bean Type 628
Broad Bean Origin 55


In [12]:
X['Bean Type'] = X['Bean Type'].replace({'\xa0' : 'Unknown'})
X['Broad Bean Origin'] = X['Broad Bean Origin'].replace({'\xa0' : 'Unknown'})
X = X.assign(Indicator1=lambda x: x['Bean Type']=='Unknown')
X['Indicator1'] = X['Indicator1'].replace({True: 1, False: 0})
X = X.assign(Indicator2=lambda x: x['Broad Bean Origin']=='Unknown')
X['Indicator2'] = X['Indicator2'].replace({True: 1, False: 0})



Сделаем маппинг Broad Bean Origin. Эта идея взята тут: https://www.kaggle.com/code/mihailodin1/preprocessing-eda-r2-0-22-baseline

In [13]:
# Словарь соответствий (дочерние -> основное). Названия регионов
region_mapping = {
    'Venezuela': ['Ven.', 'Ven', 'Venez', 'Venezuela/ Ghana', ],
    'Dominican Republic': ['Domincan Republic', 'Dom. Rep', 'D.R.', 'Domin. Rep', 'Dom. Rep.', 'Dominican Rep.', 'DR'],
    'Costa Rica': ['Cost Rica'],
    'Ecuador': ['Ecuad.', 'Ecu.'],
    'Nicaragua': ['Nic.'],
    'Madagascar': ['Mad.', 'Mad'],
    'Brazil': ['Brasil'],
    'Mexico': ['Mex'],
    'Guatemala': ['Guat.'],
    'Sao Tome & Principe': ['Sao Tome', 'Principe'],
    'Papua New Guinea': ['PNG'],
    'Carribean': ['Carribean(DR/Jam/Tri)'],
    'Indonesia': ['Java', 'Bali'],
    'West Africa': ['Africa'],
    'Trinidad': ['Trinidad-Tobago', 'Trinidad']
}

# Функция для замены названий
def standardize_region(region):
    for main_region, aliases in region_mapping.items():
        if region in aliases:
            return main_region
    return region # Если нет в словаре, оставить как есть
X['Broad Bean Origin'] = X['Broad Bean Origin'].map(standardize_region)

Добавим энкодер.

In [14]:
from category_encoders.target_encoder import TargetEncoder

te = TargetEncoder()
te.fit(X, y)
X = te.transform(X)
X.head()

,Company,REF,Cocoa Percent,Bean Type,Broad Bean Origin,Indicator1,Indicator2
0,3.215090,457,72.0,3.257923,3.236111,0,0
1,3.222739,508,72.0,3.128981,3.236111,1,0
2,3.178245,1554,70.0,3.128981,3.194636,1,0
3,3.256879,1125,72.0,3.257923,3.189583,0,0
4,3.287439,32,75.0,3.185094,3.229449,0,0


In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

Оформим все это в единой функции.

In [16]:

def data_transform(X):
    X['Cocoa Percent'] = X['Cocoa Percent'].map(lambda x: x[:-1])
    X['Cocoa Percent'] = X['Cocoa Percent'].astype(dtype='float64')
    X.drop(columns=['Company Location', 'Specific Bean Origin'], inplace=True)
    X.drop(columns=['Review'], inplace=True)
    X['Bean Type'] = X['Bean Type'].replace({'\xa0' : 'Unknown'})
    X['Broad Bean Origin'] = X['Broad Bean Origin'].replace({'\xa0' : 'Unknown'})
    X = X.assign(Indicator1=lambda x: x['Bean Type']=='Unknown')
    X['Indicator1'] = X['Indicator1'].replace({True: 1, False: 0})
    X = X.assign(Indicator2=lambda x: x['Broad Bean Origin']=='Unknown')
    X['Indicator2'] = X['Indicator2'].replace({True: 1, False: 0})
    X['Broad Bean Origin'] = X['Broad Bean Origin'].map(standardize_region)
    X = te.transform(X)
    X = scaler.transform(X)
    return X
    

In [17]:
!pip install catboost -q

С помощью оптуны подберем гиперпараметры.

In [18]:
!pip install optuna -q

import optuna
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor

def objective_cat(trial):    
    max_depth = trial.suggest_int("max_depth", 2, 16)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1, log=True)
    n_estimators = trial.suggest_int("n_estimators", 100, 2000)
    l2_leaf_reg = trial.suggest_float("l2_leaf_reg", 1, 10)
    random_strength = trial.suggest_float("random_strength", 0, 10)
    bagging_temperature = trial.suggest_float("bagging_temperature", 0, 1)
    score = cross_val_score(CatBoostRegressor(
    early_stopping_rounds = 50, max_depth=max_depth, learning_rate=learning_rate, n_estimators=n_estimators, l2_leaf_reg=l2_leaf_reg, random_strength=random_strength, bagging_temperature=bagging_temperature, verbose=100),
                            X, y, cv=3, scoring='r2').mean()
    return score

study = optuna.create_study(direction="maximize")
study.optimize(objective_cat, n_trials=150)

[I 2025-02-19 16:43:10,240] A new study created in memory with name: no-name-622143ed-4666-483d-a9b9-c83e60bd177d


0:	learn: 0.4568075	total: 53.8ms	remaining: 12.6s
100:	learn: 0.2711963	total: 83.2ms	remaining: 111ms
200:	learn: 0.2287833	total: 113ms	remaining: 19.7ms
235:	learn: 0.2189530	total: 123ms	remaining: 0us
0:	learn: 0.4563318	total: 443us	remaining: 104ms
100:	learn: 0.2741215	total: 28.9ms	remaining: 38.6ms
200:	learn: 0.2363429	total: 57.4ms	remaining: 9.99ms
235:	learn: 0.2266966	total: 67.7ms	remaining: 0us


[I 2025-02-19 16:43:10,705] Trial 0 finished with value: 0.4515086190729108 and parameters: {'max_depth': 4, 'learning_rate': 0.23784404560697434, 'n_estimators': 236, 'l2_leaf_reg': 6.250808015666442, 'random_strength': 5.95187194899384, 'bagging_temperature': 0.5827522944320878}. Best is trial 0 with value: 0.4515086190729108.


0:	learn: 0.4362118	total: 442us	remaining: 104ms
100:	learn: 0.2805617	total: 28.9ms	remaining: 38.6ms
200:	learn: 0.2382962	total: 57.3ms	remaining: 9.98ms
235:	learn: 0.2267863	total: 68.4ms	remaining: 0us
0:	learn: 0.4843267	total: 511ms	remaining: 16m 51s
100:	learn: 0.2696389	total: 43.9s	remaining: 13m 37s
200:	learn: 0.2060842	total: 1m 26s	remaining: 12m 50s
300:	learn: 0.1712113	total: 2m 12s	remaining: 12m 19s
400:	learn: 0.1482550	total: 2m 57s	remaining: 11m 40s
500:	learn: 0.1301238	total: 3m 41s	remaining: 10m 54s
600:	learn: 0.1153867	total: 4m 19s	remaining: 9m 56s
700:	learn: 0.1054916	total: 4m 58s	remaining: 9m 5s
800:	learn: 0.0958797	total: 5m 41s	remaining: 8m 22s
900:	learn: 0.0874965	total: 6m 23s	remaining: 7m 40s
1000:	learn: 0.0794798	total: 7m 2s	remaining: 6m 54s
1100:	learn: 0.0732512	total: 7m 42s	remaining: 6m 10s
1200:	learn: 0.0678814	total: 8m 22s	remaining: 5m 27s
1300:	learn: 0.0625061	total: 9m 3s	remaining: 4m 44s
1400:	learn: 0.0580381	total: 9m

[I 2025-02-19 17:26:16,344] Trial 1 finished with value: 0.31827588841583376 and parameters: {'max_depth': 16, 'learning_rate': 0.04605480931686801, 'n_estimators': 1982, 'l2_leaf_reg': 9.539403216436824, 'random_strength': 0.1469554926814043, 'bagging_temperature': 0.5021321746104142}. Best is trial 0 with value: 0.4515086190729108.


...

0:	learn: 0.4868224	total: 453us	remaining: 616ms
100:	learn: 0.3991839	total: 29.5ms	remaining: 368ms
200:	learn: 0.3682814	total: 59.9ms	remaining: 345ms
300:	learn: 0.3513613	total: 90.4ms	remaining: 317ms
400:	learn: 0.3417132	total: 119ms	remaining: 283ms
500:	learn: 0.3351104	total: 147ms	remaining: 252ms
600:	learn: 0.3295343	total: 176ms	remaining: 221ms
700:	learn: 0.3242724	total: 204ms	remaining: 191ms
800:	learn: 0.3193182	total: 234ms	remaining: 163ms
900:	learn: 0.3149172	total: 264ms	remaining: 134ms
1000:	learn: 0.3100451	total: 293ms	remaining: 104ms
1100:	learn: 0.3061652	total: 323ms	remaining: 75.4ms
1200:	learn: 0.3027622	total: 352ms	remaining: 45.9ms
1300:	learn: 0.2995049	total: 380ms	remaining: 16.6ms
1357:	learn: 0.2975879	total: 397ms	remaining: 0us
0:	learn: 0.4816506	total: 446us	remaining: 606ms
100:	learn: 0.4029089	total: 29.8ms	remaining: 371ms
200:	learn: 0.3714412	total: 59.4ms	remaining: 342ms
300:	learn: 0.3569761	total: 88.2ms	remaining: 310ms
400:

[I 2025-02-19 17:57:26,537] Trial 148 finished with value: 0.4729814832052171 and parameters: {'max_depth': 4, 'learning_rate': 0.011847420194852352, 'n_estimators': 1358, 'l2_leaf_reg': 9.253231008157343, 'random_strength': 9.312500363892715, 'bagging_temperature': 0.5679559743340366}. Best is trial 32 with value: 0.48049165155846185.


1300:	learn: 0.3048450	total: 347ms	remaining: 15.2ms
1357:	learn: 0.3032156	total: 364ms	remaining: 0us
0:	learn: 0.4828160	total: 331us	remaining: 594ms
100:	learn: 0.3546336	total: 18ms	remaining: 301ms
200:	learn: 0.3365954	total: 34.7ms	remaining: 275ms
300:	learn: 0.3246226	total: 52.6ms	remaining: 261ms
400:	learn: 0.3176691	total: 69.5ms	remaining: 242ms
500:	learn: 0.3125815	total: 86.9ms	remaining: 224ms
600:	learn: 0.3081366	total: 106ms	remaining: 210ms
700:	learn: 0.3040861	total: 123ms	remaining: 192ms
800:	learn: 0.3004073	total: 139ms	remaining: 173ms
900:	learn: 0.2968520	total: 154ms	remaining: 153ms
1000:	learn: 0.2936417	total: 169ms	remaining: 134ms
1100:	learn: 0.2904258	total: 183ms	remaining: 115ms
1200:	learn: 0.2874975	total: 197ms	remaining: 97.4ms
1300:	learn: 0.2846171	total: 215ms	remaining: 81.3ms
1400:	learn: 0.2820863	total: 233ms	remaining: 65.3ms
1500:	learn: 0.2795724	total: 250ms	remaining: 48.8ms
1600:	learn: 0.2773233	total: 267ms	remaining: 32.2m

[I 2025-02-19 17:57:27,718] Trial 149 finished with value: 0.4721254029952928 and parameters: {'max_depth': 2, 'learning_rate': 0.04005861139207635, 'n_estimators': 1794, 'l2_leaf_reg': 9.515079358304172, 'random_strength': 5.819649620248434, 'bagging_temperature': 0.16796208669224988}. Best is trial 32 with value: 0.48049165155846185.


1100:	learn: 0.2998898	total: 184ms	remaining: 116ms
1200:	learn: 0.2969350	total: 200ms	remaining: 98.6ms
1300:	learn: 0.2942160	total: 216ms	remaining: 81.9ms
1400:	learn: 0.2917766	total: 233ms	remaining: 65.3ms
1500:	learn: 0.2894645	total: 249ms	remaining: 48.6ms
1600:	learn: 0.2871039	total: 266ms	remaining: 32.1ms
1700:	learn: 0.2847866	total: 283ms	remaining: 15.5ms
1793:	learn: 0.2828386	total: 298ms	remaining: 0us


In [19]:

from catboost import CatBoostRegressor
model = CatBoostRegressor(**study.best_params)
model.fit(X, y)


0:	learn: 0.4725957	total: 493us	remaining: 451ms
1:	learn: 0.4661270	total: 1.19ms	remaining: 545ms
2:	learn: 0.4599076	total: 1.63ms	remaining: 496ms
3:	learn: 0.4545614	total: 2.09ms	remaining: 477ms
4:	learn: 0.4497277	total: 2.53ms	remaining: 461ms
5:	learn: 0.4447651	total: 2.96ms	remaining: 448ms
6:	learn: 0.4417325	total: 3.35ms	remaining: 435ms
7:	learn: 0.4381342	total: 3.77ms	remaining: 428ms
8:	learn: 0.4352000	total: 4.19ms	remaining: 422ms
9:	learn: 0.4313092	total: 4.58ms	remaining: 415ms
10:	learn: 0.4278495	total: 5ms	remaining: 411ms
11:	learn: 0.4247805	total: 5.43ms	remaining: 409ms
12:	learn: 0.4209983	total: 5.82ms	remaining: 403ms
13:	learn: 0.4181501	total: 6.25ms	remaining: 402ms
14:	learn: 0.4161019	total: 6.59ms	remaining: 395ms
15:	learn: 0.4134237	total: 6.98ms	remaining: 392ms
16:	learn: 0.4124633	total: 7.42ms	remaining: 392ms
17:	learn: 0.4116747	total: 7.83ms	remaining: 390ms
18:	learn: 0.4093686	total: 8.22ms	remaining: 388ms
19:	learn: 0.4062708	total

In [20]:
X = pd.read_csv('/kaggle/input/chocolate-rating-prediction-ai-edu/chocolate_test_new.csv')
X

,Company,Specific Bean Origin,REF,Review,Cocoa Percent,Company Location,Bean Type,Broad Bean Origin
0,Ohiyo,"San Juan Estate, Gran Couva",1594,2015,70%,U.S.A.,Trinitario,Trinidad
1,Blanxart,Organic Dark,322,2009,72%,Spain,,
2,Brazen,Maya Mountain,1518,2015,80%,U.S.A.,Trinitario,Belize
3,Patric,Madagascar,439,2009,67%,U.S.A.,Trinitario,Madagascar
4,Potomac,"Upala, Batch 12",607,2010,82%,U.S.A.,Matina,Costa Rica
...,...,...,...,...,...,...,...,...
535,Monsieur Truffe,Papaua New Guinea,1391,2014,70%,Australia,,Papua New Guinea
536,Cacao Hunters,Macondo,1816,2016,71%,Colombia,,Colombia
537,Bonnat,Maragnam,1038,2013,75%,France,,Brazil
538,Altus aka Cao Artisan,Acopagro,1728,2016,60%,U.S.A.,,Peru


In [21]:
X = data_transform(X)
pred = model.predict(X)
result = pd.DataFrame()
result['id'] = np.arange(540)
result['Rating'] = pred
result[['id','Rating']].to_csv("submission.csv", index=False)